In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install bar_chart_race

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 3.5 MB/s eta 0:00:00


In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import bar_chart_race as bcr
import cv2
import numpy as np

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FusionModel(nn.Module):
    def __init__(self):
        super(FusionModel, self).__init__()
        self.relu=nn.ReLU()    # inplace=True 삭제, error 뜸
        ### optical layer에 sigmoid 함수 필요해서 추가함 + fusion and pooling 단계 maxpool3d도
        self.sigmoid = nn.Sigmoid()
        self.maxpool = nn.MaxPool3d(kernel_size=(8, 1 ,1), stride=(8, 1, 1))

        ## Hint: Please refer to above table for constructing layers
        # Construct block of RGB layers which takes RGB channel(3) as input
        self.rgb_conv1 = nn.Conv3d(3, 16, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
        self.rgb_conv2 = nn.Conv3d(16, 16, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
        self.rgb_maxpool1 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))
        self.rgb_conv3 = nn.Conv3d(16, 16, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
        self.rgb_conv4 = nn.Conv3d(16, 16, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
        self.rgb_maxpool2 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))

        self.rgb_conv5 = nn.Conv3d(16, 32, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
        self.rgb_conv6 = nn.Conv3d(32, 32, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
        self.rgb_maxpool3 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))
        self.rgb_conv7 = nn.Conv3d(32, 32, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
        self.rgb_conv8 = nn.Conv3d(32, 32, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
        self.rgb_maxpool4 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))

        # Construct block of optical flow layers which takes the optical flow channel(2) as input
        self.opt_conv1 = nn.Conv3d(2, 16, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
        self.opt_conv2 = nn.Conv3d(16, 16, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
        self.opt_maxpool1 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))
        self.opt_conv3 = nn.Conv3d(16, 16, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
        self.opt_conv4 = nn.Conv3d(16, 16, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
        self.opt_maxpool2 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))

        self.opt_conv5 = nn.Conv3d(16, 32, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
        self.opt_conv6 = nn.Conv3d(32, 32, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
        self.opt_maxpool3 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))
        self.opt_conv7 = nn.Conv3d(32, 32, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
        self.opt_conv8 = nn.Conv3d(32, 32, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
        self.opt_maxpool4 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))

        # Construct merging Block
        self.merge_conv1 = nn.Conv3d(32, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
        self.merge_conv2 = nn.Conv3d(64, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
        self.merge_maxpool1 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))
        self.merge_conv3 = nn.Conv3d(64, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
        self.merge_conv4 = nn.Conv3d(64, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
        self.merge_maxpool2 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))

        self.merge_conv5 = nn.Conv3d(64, 128, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
        self.merge_conv6 = nn.Conv3d(128, 128, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
        self.merge_maxpool3 = nn.MaxPool3d(kernel_size=(2, 3, 3), stride=(2, 3, 3))

        ## 얘는 주어진 코드
        # Fully Connected Layers
        self.fc1 = nn.Linear(128, 128)
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(128, 32)
        self.fc3 = nn.Linear(32, 2)

        # Initialize weights
        self.__init_weight()

    def forward(self, x):
        x = x.transpose(2, 4)
        x = x.transpose(3, 4)
        x = x.transpose(1, 2)
        rgb = x[:,:3,:,:,:]
        opt = x[:,3:5,:,:,:]

        # Pass through the RGB data through the blocks of RGB layers
        rgb = self.rgb_conv1(rgb)
        rgb = self.relu(rgb)
        rgb = self.rgb_conv2(rgb)
        rgb = self.relu(rgb)
        rgb = self.rgb_maxpool1(rgb)
        rgb = self.rgb_conv3(rgb)
        rgb = self.relu(rgb)
        rgb = self.rgb_conv4(rgb)
        rgb = self.relu(rgb)
        rgb = self.rgb_maxpool2(rgb)

        rgb = self.rgb_conv5(rgb)
        rgb = self.relu(rgb)
        rgb = self.rgb_conv6(rgb)
        rgb = self.relu(rgb)
        rgb = self.rgb_maxpool3(rgb)
        rgb = self.rgb_conv7(rgb)
        rgb = self.relu(rgb)
        rgb = self.rgb_conv8(rgb)
        rgb = self.relu(rgb)
        rgb = self.rgb_maxpool4(rgb)

        # Pass through the optical flow data through the blocks of RGB layers
        opt = self.opt_conv1(opt)
        opt = self.relu(opt)
        opt = self.opt_conv2(opt)
        opt = self.relu(opt)
        opt = self.opt_maxpool1(opt)
        opt = self.opt_conv3(opt)
        opt = self.relu(opt)
        opt = self.opt_conv4(opt)
        opt = self.relu(opt)
        opt = self.opt_maxpool2(opt)

        opt = self.opt_conv5(opt)
        opt = self.relu(opt)
        opt = self.opt_conv6(opt)
        opt = self.relu(opt)
        opt = self.opt_maxpool3(opt)
        opt = self.opt_conv7(opt)
        opt = self.sigmoid(opt)
        opt = self.opt_conv8(opt)
        opt = self.sigmoid(opt)
        opt = self.opt_maxpool4(opt)

        # Fuse by performing elementwise multiplication of rgb and opt tensors.
        #fused = ## fill here ##
        fused = rgb * opt
        # Perform maxpooling of fused
        #fused = ## fill here ##
        fused = self.maxpool(fused)

        # Pass through the fused data into merging block
        merged = self.merge_conv1(fused)
        merged = self.relu(merged)
        merged = self.merge_conv2(merged)
        merged = self.relu(merged)
        merged = self.merge_maxpool1(merged)
        merged = self.merge_conv3(merged)
        merged = self.relu(merged)
        merged = self.merge_conv4(merged)
        merged = self.relu(merged)
        merged = self.merge_maxpool2(merged)
        merged = self.merge_conv5(merged)
        merged = self.relu(merged)
        merged = self.merge_conv6(merged)
        merged = self.relu(merged)
        merged = self.merge_maxpool3(merged)

        # Fully Connected Layers
        x = merged.view(merged.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)

        return x

    def __init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                # Perform weight initialization ("kaiming normal")
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    #nn.init.constant_(m.bias, 0)
                    fan_in, _ = nn.init._calculate_fan_in_and_fan_out(m.weight)
                    bound = 1 / torch.sqrt(torch.tensor(fan_in))
                    nn.init.uniform_(m.bias, -bound, bound)

In [ ]:
class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer = nn.Linear(64*224*224*5, 2)

  def forward(self, x):
    x = x.view(x.shape[0], -1)
    x = self.layer(x)
    return x

In [ ]:
device = torch.device('cuda')

model = FusionModel().to(device)
#model = Model().to(device)
# user your pretrained model path
#model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/Computer Vision/AnomalyDetection/model_weight_0.01.pth'))
model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/Computer Vision/AnomalyDetection/model_weight.pth'))

<All keys matched successfully>

In [ ]:
def uniform_sampling(video, target_frames=64):
    # get total frames of input video and calculate sampling interval
    len_frames = int(len(video))
    interval = int(np.ceil(len_frames/target_frames))
    # init empty list for sampled video and
    sampled_video = []
    for i in range(0,len_frames,interval):
        sampled_video.append(video[i])
    # calculate numer of padded frames and fix it
    num_pad = target_frames - len(sampled_video)
    padding = []
    if num_pad>0:
        for i in range(-num_pad,0):
            try:
                padding.append(video[i])
            except:
                padding.append(video[0])
        sampled_video += padding
    # get sampled video
    return np.array(sampled_video, dtype=np.float32)


def normalize(data):
    mean = data.mean()
    std = data.std()
    return (data - mean) / std

In [ ]:
from tqdm.auto import tqdm

# test video path
file_path ="/content/drive/MyDrive/Colab Notebooks/Computer Vision/AnomalyDetection/Assault018_x264.mp4"

cap = cv2.VideoCapture(file_path)
# Get number of frames
len_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

frames = []
flows = []
preds = []
resize = (224, 224)

prev_frame = None
for iter in tqdm(range(0, len_frames-1)):
    _, frame = cap.read()
    frame = cv2.resize(frame, resize, interpolation=cv2.INTER_AREA)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = np.reshape(frame, (224, 224, 3))
    frames.append(frame)

    img = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    gray_img = np.reshape(img, (224, 224, 1))
    if prev_frame is None:
        prev_frame = gray_img

    flow = cv2.calcOpticalFlowFarneback(prev_frame, gray_img, None, 0.5, 3, 15, 3, 5, 1.2,
                                        cv2.OPTFLOW_FARNEBACK_GAUSSIAN)
    prev_frame = gray_img
    # subtract the mean in order to eliminate the movement of camera
    flow[..., 0] -= np.mean(flow[..., 0])
    flow[..., 1] -= np.mean(flow[..., 1])
    # normalize each component in optical flow
    flow[..., 0] = cv2.normalize(flow[..., 0], None, 0, 255, cv2.NORM_MINMAX)
    flow[..., 1] = cv2.normalize(flow[..., 1], None, 0, 255, cv2.NORM_MINMAX)

    flows.append(flow)
    result = np.zeros((len(flows), 224, 224, 5))
    result[..., :3] = frames
    result[..., 3:] = flows

    data = np.float32(result)
    # # sampling 64 frames uniformly from the entire video
    data = uniform_sampling(video=data, target_frames=64)
    # normalize rgb images and optical flows, respectively
    data[..., :3] = normalize(data[..., :3])
    data[..., 3:] = normalize(data[..., 3:])

    fr, w, h, ch = data.shape
    data = data.reshape((-1, fr, w, h, ch))
    # pred = model.predict(data)[0]
    pred = model(torch.Tensor(data).to(device))
    # fights.append(pred[0])
    # none.append(pred[1])
    preds.append(pred[0].detach().cpu().numpy())

cap.release()

  0%|          | 0/360 [00:00<?, ?it/s]

In [ ]:
import pandas as pd

print(np.array(preds).shape)

group_list = ["Violence", "Non-Violence"]
df = pd.DataFrame(preds, columns = ['Violence', 'Non-Violence'])

df

(360, 2)


,Violence,Non-Violence
0,-0.185850,0.352300
1,-0.179158,0.354360
2,-0.179938,0.351959
3,-0.179374,0.351487
4,-0.182000,0.349516
...,...,...
355,-0.094814,0.273985
356,-0.036173,0.187575
357,-0.104893,0.275145
358,-0.077687,0.238162


In [ ]:
bcr.bar_chart_race(df=df[:],
                   n_bars = 2,
                   figsize=(4, 4),
                   label_bars=False,
                   sort='desc',
                   #title='Violence detection',
                   fixed_order=['Violence', 'Non-Violence'],
                   orientation='h',
                   fixed_max=True,
                   period_length=int(1000/fps),
                  )